In [2]:
%load_ext autoreload
%autoreload 2
from ambiguous.dataset.dataset import *

# EMNIST example

In [ ]:
save_dataset_to_file(dataset_name='EMNIST',
                     og_root='/share/datasets/',
                     new_root='/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST/',
                     blend=0.5) # can specify pairs as argument, otherwise uses default from dataset.py

In [ ]:
trainset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST', train=True)
testset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/EMNIST', train=False)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)
print(len(trainset), len(testset))
x, t = next(iter(trainloader))
i = 0
x.shape, [(t[0][i], t[1][i]) for i in range(5)]

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(make_grid(x, nrow=8).detach().cpu().permute(1,2,0))
plt.axis('off')

# MNIST example

In [ ]:
save_dataset_to_file('MNIST', '/share/datasets/', '/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/',blend=0.5)

In [ ]:
trainset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/', train=True)
testset = DatasetFromNPY('/home/nislah/ambiguous-dataset/ambiguous/data/MNIST/', train=False)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=True)
print(len(trainset), len(testset))
x, t = next(iter(trainloader))
i = 0
x.shape, t[0][i], t[1][i]

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(make_grid(x, nrow=8).detach().cpu().permute(1,2,0))
plt.axis('off')

# new EMNIST (Conv conditional)

In [3]:
%load_ext autoreload
%autoreload 2
# Import dependencies
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import torchvision
import seaborn as sns
import numpy as np
import pytorch_lightning as pl
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.utils import save_image
import torch
from tqdm import tqdm
from datetime import datetime
import yaml
import h5py
from copy import deepcopy
import ambiguous.models.cvae
from ambiguous.models.cvae import *
from ambiguous.dataset.dataset import partition_dataset
import wandb
from pytorch_lightning.loggers import WandbLogger

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Seed for reproducibility
torch.manual_seed(42)
print(device)
root='/home/mila/n/nizar.islah/expectation-clamp/'
transform = transforms.Compose([
    transforms.ToTensor(),
    lambda x: x.rot90(1,[2,1]).flip(2)
])
dataset = datasets.EMNIST(root=root, download=False, split='byclass', train=True, transform=transform)
new_dataset = partition_dataset(dataset, range(10, 36))
train_set, val_set = torch.utils.data.random_split(new_dataset, [round(0.8*len(new_dataset)), round(0.2*len(new_dataset))])
test_set = datasets.EMNIST(root=root, download=False, split='byclass', train=False, transform=transform)
# Dataloaders
batch_size = 256
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=2, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=2)

cuda


In [5]:
model = torch.load('../train/ConvCVAE.pth').to(device)

In [6]:
n_cls=26
img_size=28
lr=1e-3
num_epochs=50
latent_dim = 20

onehot = torch.zeros(n_cls, n_cls).to(device)
onehot = onehot.scatter_(1, torch.LongTensor(range(n_cls)).view(n_cls,1).to(device), 1).view(n_cls, n_cls, 1, 1)
fill = torch.zeros([n_cls, n_cls, img_size, img_size]).to(device)
for i in range(n_cls):
    fill[i, i, :, :] = 1

In [23]:
model.eval()
for _, (images, labels) in enumerate(val_loader):
    with torch.no_grad():
        images = images.to(device)
        labels = labels.to(device)
        labels_fill_ = fill[labels]
        y_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
        y_label_ = onehot[y_]
        y2_ = (torch.rand(images.size(0), 1) * n_cls).type(torch.LongTensor).squeeze()
        y2_label_ = onehot[y2_]
        y_label = (y_label_ + y2_label_)/2
        print(y_label)
        rec, mu, logvar = model((images, labels_fill_, y_label))
    break

tensor([[[[0.0000]],

         [[0.5000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]],


        [[[0.0000]],

         [[0.0000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]],


        [[[0.0000]],

         [[0.0000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]],


        ...,


        [[[0.5000]],

         [[0.0000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]],


        [[[0.0000]],

         [[0.0000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]],


        [[[0.0000]],

         [[0.0000]],

         [[0.0000]],

         ...,

         [[0.0000]],

         [[0.0000]],

         [[0.0000]]]], device='cuda:0')


In [29]:
torchvision.utils.save_image(rec, "example_mix.pdf")

In [30]:
y_label[0]

tensor([[[0.0000]],

        [[0.5000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]],

        [[0.5000]],

        [[0.0000]],

        [[0.0000]],

        [[0.0000]]], device='cuda:0')

In [10]:
pwd

'/home/mila/n/nizar.islah/ambiguous-dataset/ambiguous/dataset'

In [11]:
readout = torch.load('/home/mila/n/nizar.islah/expectation-clamp/results/conv_vae_20220504_133037/readout.pt').to(device)

ModuleNotFoundError: No module named 'models'

In [12]:
vae = torch.load('/home/mila/n/nizar.islah/expectation-clamp/results/conv_vae_20220504_124933/model.pt').to(device)

ModuleNotFoundError: No module named 'models'